In [3]:
import subprocess
import numpy as np
import healpy as hp
import pickle
from random import randint
import joblib
import sklearn
import sys
import importlib

sys.path.append('../methods_code_Nresol/')
sys.path.append('../utils/')
from utils_circpatch import *
from methods2 import *
import vis_utils
import astro_cuts
importlib.reload(astro_cuts)

<module 'astro_cuts' from '/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/astro_cuts.py'>

In [4]:
#are all b>21 tiles there?
fullsky = vis_utils.get_testbed_dict('FullSky_Bayestar')
numtiles = np.unique(get_largepix_for_smallpix(fullsky['coords'], 2048, 32))
lat = hp.pix2ang(32, numtiles, lonlat=True)[1]
bgt20mask = np.abs(lat)>21
t32bgt20 = numtiles[bgt20mask]
print(len(t32bgt20))

6244


In [6]:
runname = 'fullsky_runs/16a_fwhm-6-1_'
recontiles = []
for tile in t32bgt20:
    if os.path.exists(runname+'_tmpdir/recon_{}.hdf5'.format(tile)):
        recontiles.append(tile)
len(recontiles)

6243

In [7]:
wisemodel = joblib.load('spectra_matched/models/svm_panstarrs_quasarsep_balanced.joblib')
smodel = joblib.load('spectra_matched/models/svm_sdss_quasarsep_unbalanced.joblib')

cuts_list = [('distmod_median_cut', {'mindm': 8}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut', {'model': wisemodel}), ('parallax_nan_edr3',),
             ('sdss_uvcut', {'model': smodel}), ('bayestar_chisq', {'maxchisq': 3.0})]
outer_kwargs = {'Nsidetile': 32, 'radius_deg_extra': 25/60, 'Nsideresol': 2048,
               'stars_presaved': '/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits'}
recon_kwargs = {'length': np.deg2rad(6.1/(2.355*60)), 'scale': 1.0, 'no_invvar': False}
kwargs_dict = {'outer_kwargs': outer_kwargs, 'recon_kwargs': recon_kwargs}
recon_func_name = 'gaussiankernel_reconstruction_effective_spherical3_blocks'

In [8]:
reconmap, varmap, reconpix = get_Nsideresol_healpix_map_from_gnuparpatches_safe(runname+'_tmpdir/', Nsideresol=kwargs_dict['outer_kwargs']['Nsideresol'], tiles=recontiles)
result_dict = {'dustmap': reconmap, 'variancemap': varmap, 'reconpix': reconpix,
               'func': 'gnupartilewise_new',
               'cuts_list': cuts_list, 'recon_func_name': recon_func_name,
               'kwargs_dict': kwargs_dict}
pickle.dump(result_dict, open(runname + '_bgt21.pkl', 'wb'))      

Ckp
